<a href="https://colab.research.google.com/github/HanSeungJun/2021_study/blob/main/22~24_DataScience_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터사이언스 2022~2024 집행내역 데이터 가공 및 취합

In [2]:
import zipfile
import pandas as pd
import os

# zip 파일 압축 해제
with zipfile.ZipFile('master.zip', 'r') as zip_ref:
    zip_ref.extractall('master')

# 압축 해제된 파일 목록 확인
extracted_files = os.listdir('master')
print("압축 해제된 파일 목록:", extracted_files)

# 올바른 파일 이름으로 경로 수정
excel_file_path = 'master/집행/22년도 데사_고려대.xlsx'  # 파일 경로를 실제 경로에 맞게 수정하세요.

# 엑셀 파일 로드
try:
    df = pd.read_excel(excel_file_path)
    # 엑셀 파일의 헤드 부분 출력
    print(df.head())
except FileNotFoundError as e:
    print(f"파일을 찾을 수 없습니다: {e}")


압축 해제된 파일 목록: ['집행']
    No       결의일자       이체일자 카드사용일자         항목                사용용도  집행방법  \
0  NaN        NaT        NaT    NaT        NaN                 NaN   NaN   
1  1.0 2022-05-27 2022-05-27    NaT        간접비            간접비(비영리)  계좌이체   
2  2.0 2022-06-15 2022-06-15    NaT  (특례)학생인건비               학생인건비  계좌이체   
3  3.0 2022-06-21 2022-06-24    NaT        인건비  비영리법인연구부서연구근접지원인력비  계좌이체   
4  4.0 2022-06-21 2022-06-24    NaT        인건비  비영리법인연구부서연구근접지원인력비  계좌이체   

          공급가액  부가세      연구비사용금액  ...  연구수행기관 자체검토내역 Unnamed: 27 Unnamed: 28  \
0          NaN  NaN          NaN  ...           보완상태        검토결과        인정금액   
1  215542810.0  0.0  215542810.0  ...            NaN         NaN           0   
2  108800000.0  0.0  108800000.0  ...            NaN         NaN           0   
3    2500000.0  0.0    2500000.0  ...            NaN         NaN           0   
4     564520.0  0.0     564520.0  ...            NaN         NaN           0   

  Unnamed: 29 Unnamed: 30 Unnamed: 31 Unn

In [4]:
import pandas as pd

# 파일 경로
file_path = excel_file_path

# 엑셀 파일 읽기
xls = pd.ExcelFile(file_path)

# 첫 번째 시트 불러오기 (시트명이 여러 개일 수 있으므로 첫 번째 시트 사용)
df = xls.parse(xls.sheet_names[0], header=None)

# 5번 행(즉, 인덱스 4)을 '항목'으로 사용
# 8번 행(즉, 인덱스 7)을 '공급가액'으로 사용
df_selected = df.iloc[[4, 7]].T  # 행을 열로 변환

df_selected.columns = ['항목', '공급가액']  # 컬럼명 설정

# '공급가액'을 숫자로 변환 (문자열이 포함될 가능성 대비)
df_selected['공급가액'] = pd.to_numeric(df_selected['공급가액'], errors='coerce')

# NaN 값 제거
df_selected = df_selected.dropna()

# '항목'별 '공급가액' 합산
result = df_selected.groupby('항목')['공급가액'].sum().reset_index()

# 결과 출력
print(result)


                  항목          공급가액
0                  0  0.000000e+00
1            2500000  3.756000e+04
2  20220000004114550  2.022000e+16
3                  3  6.000000e+00


In [6]:
# 데이터 확인용 출력
print(df.head(10))  # 상위 10개 행 출력


    0                    1                    2       3          4   \
0   No                 결의일자                 이체일자  카드사용일자         항목   
1  NaN                  NaN                  NaN     NaN        NaN   
2    1  2022-05-27 00:00:00  2022-05-27 00:00:00     NaN        간접비   
3    2  2022-06-15 00:00:00  2022-06-15 00:00:00     NaN  (특례)학생인건비   
4    3  2022-06-21 00:00:00  2022-06-24 00:00:00     NaN        인건비   
5    4  2022-06-21 00:00:00  2022-06-24 00:00:00     NaN        인건비   
6    5  2022-06-24 00:00:00  2022-06-28 00:00:00     NaN        인건비   
7    6  2022-06-24 00:00:00  2022-06-28 00:00:00     NaN        인건비   
8    7  2022-06-24 00:00:00  2022-06-28 00:00:00     NaN        인건비   
9    8  2022-06-24 00:00:00  2022-06-28 00:00:00     NaN        인건비   

                   5     6          7    8          9   ...             26  \
0                사용용도  집행방법       공급가액  부가세    연구비사용금액  ...  연구수행기관 자체검토내역   
1                 NaN   NaN        NaN  NaN        NaN  ...   

# [결과1] 22년도 데이터 사이언스 집행 (5건)

In [13]:
import pandas as pd
import glob
import os

# 폴더 경로 및 파일 검색 ("집행" 폴더 내 "22년도" 포함된 파일 찾기)
folder_path = "master/집행/"
file_pattern = os.path.join(folder_path, "*22년도*.xlsx")
file_list = glob.glob(file_pattern)

# 여러 파일에 대한 결과 저장
def process_file(file_path):
    # 엑셀 파일 읽기
    xls = pd.ExcelFile(file_path)
    df = xls.parse(xls.sheet_names[0])  # 첫 번째 시트 사용

    # '항목'과 '공급가액' 열 선택
    df_selected = df.iloc[:, [4, 7]].copy()
    df_selected.columns = ['항목', '공급가액']

    # 병합된 셀 문제 해결: NaN을 앞의 값으로 채움
    df_selected['항목'] = df_selected['항목'].fillna(method='ffill')

    # '공급가액'을 숫자로 변환
    df_selected['공급가액'] = pd.to_numeric(df_selected['공급가액'], errors='coerce')

    # NaN 값 제거 ('공급가액'이 없는 행만 삭제)
    df_selected = df_selected.dropna(subset=['공급가액'])

    # '항목'별 '공급가액' 합산
    result = df_selected.groupby('항목', as_index=False)['공급가액'].sum()

    # 파일명 추가
    result.insert(0, '파일명', os.path.basename(file_path))

    return result

# 모든 파일에 대해 처리 및 출력
for file in file_list:
    result_df = process_file(file)
    print(f"파일명: {os.path.basename(file)}")
    print(result_df)

<ipython-input-13-2f0c1d1b78bc>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')
<ipython-input-13-2f0c1d1b78bc>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')


파일명: 22년도 데사_카이스트.xlsx
                 파일명            항목         공급가액
0  22년도 데사_카이스트.xlsx  (일반)연구시설•장비비  115484000.0
1  22년도 데사_카이스트.xlsx     (특례)학생인건비  549900000.0
2  22년도 데사_카이스트.xlsx           간접비  195000000.0
3  22년도 데사_카이스트.xlsx         연구재료비    7480000.0
4  22년도 데사_카이스트.xlsx         연구활동비  132147688.0
5  22년도 데사_카이스트.xlsx           인건비   73715479.0
파일명: 22년도 데사_서울시립.xlsx
                 파일명            항목         공급가액
0  22년도 데사_서울시립.xlsx  (일반)연구시설•장비비  100827900.0
1  22년도 데사_서울시립.xlsx     (특례)학생인건비  232666000.0
2  22년도 데사_서울시립.xlsx           간접비  137408000.0
3  22년도 데사_서울시립.xlsx          연구수당   40000000.0
4  22년도 데사_서울시립.xlsx         연구활동비   97759964.0


<ipython-input-13-2f0c1d1b78bc>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')


파일명: 22년도 데사_충남대.xlsx
                파일명            항목         공급가액
0  22년도 데사_충남대.xlsx  (일반)연구시설•장비비   90850590.0
1  22년도 데사_충남대.xlsx     (특례)학생인건비  111000000.0
2  22년도 데사_충남대.xlsx           간접비  114630000.0
3  22년도 데사_충남대.xlsx          연구수당   47080000.0
4  22년도 데사_충남대.xlsx         연구활동비  213709211.0
5  22년도 데사_충남대.xlsx           인건비   36253490.0


<ipython-input-13-2f0c1d1b78bc>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')


파일명: 22년도 데사_고려대.xlsx
                파일명            항목         공급가액
0  22년도 데사_고려대.xlsx  (일반)연구시설•장비비  160419800.0
1  22년도 데사_고려대.xlsx     (특례)학생인건비  108800000.0
2  22년도 데사_고려대.xlsx           간접비  272839000.0
3  22년도 데사_고려대.xlsx          연구수당   46000000.0
4  22년도 데사_고려대.xlsx         연구활동비  145017692.0
5  22년도 데사_고려대.xlsx           인건비   28784564.0
파일명: 22년도 데사_호서대.xlsx
                파일명            항목         공급가액
0  22년도 데사_호서대.xlsx  (일반)연구시설•장비비  126974000.0
1  22년도 데사_호서대.xlsx     (일반)학생인건비   69140000.0
2  22년도 데사_호서대.xlsx           간접비  138550000.0
3  22년도 데사_호서대.xlsx          연구수당   45868000.0
4  22년도 데사_호서대.xlsx         연구활동비  168312964.0
5  22년도 데사_호서대.xlsx           인건비   15866820.0


<ipython-input-13-2f0c1d1b78bc>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')


# [결과2] 23년도 데이터 사이언스 집행 (10건)

In [14]:
import pandas as pd
import glob
import os

# 폴더 경로 및 파일 검색 ("집행" 폴더 내 "22년도" 포함된 파일 찾기)
folder_path = "master/집행/"
file_pattern = os.path.join(folder_path, "*23년도*.xlsx")
file_list = glob.glob(file_pattern)

# 여러 파일에 대한 결과 저장
def process_file(file_path):
    # 엑셀 파일 읽기
    xls = pd.ExcelFile(file_path)
    df = xls.parse(xls.sheet_names[0])  # 첫 번째 시트 사용

    # '항목'과 '공급가액' 열 선택
    df_selected = df.iloc[:, [4, 7]].copy()
    df_selected.columns = ['항목', '공급가액']

    # 병합된 셀 문제 해결: NaN을 앞의 값으로 채움
    df_selected['항목'] = df_selected['항목'].fillna(method='ffill')

    # '공급가액'을 숫자로 변환
    df_selected['공급가액'] = pd.to_numeric(df_selected['공급가액'], errors='coerce')

    # NaN 값 제거 ('공급가액'이 없는 행만 삭제)
    df_selected = df_selected.dropna(subset=['공급가액'])

    # '항목'별 '공급가액' 합산
    result = df_selected.groupby('항목', as_index=False)['공급가액'].sum()

    # 파일명 추가
    result.insert(0, '파일명', os.path.basename(file_path))

    return result

# 모든 파일에 대해 처리 및 출력
for file in file_list:
    result_df = process_file(file)
    print(f"파일명: {os.path.basename(file)}")
    print(result_df)

<ipython-input-14-8ad0d7f27e0c>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')
<ipython-input-14-8ad0d7f27e0c>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')


파일명: 23년도 데사_카이스트.xlsx
                 파일명            항목         공급가액
0  23년도 데사_카이스트.xlsx  (일반)연구시설•장비비  158821560.0
1  23년도 데사_카이스트.xlsx     (특례)학생인건비  769860000.0
2  23년도 데사_카이스트.xlsx           간접비  274950000.0
3  23년도 데사_카이스트.xlsx          연구수당   53040000.0
4  23년도 데사_카이스트.xlsx         연구재료비   53497770.0
5  23년도 데사_카이스트.xlsx         연구활동비  138474437.0
6  23년도 데사_카이스트.xlsx           인건비   99875041.0
파일명: 23년도 데사_부산대.xlsx
                파일명            항목         공급가액
0  23년도 데사_부산대.xlsx  (일반)연구시설•장비비   93562520.0
1  23년도 데사_부산대.xlsx           간접비   51136000.0
2  23년도 데사_부산대.xlsx          연구수당   20000000.0
3  23년도 데사_부산대.xlsx         연구재료비    3999000.0
4  23년도 데사_부산대.xlsx         연구활동비  159792546.0
5  23년도 데사_부산대.xlsx           인건비   15353150.0


<ipython-input-14-8ad0d7f27e0c>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')
<ipython-input-14-8ad0d7f27e0c>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')


파일명: 23년도 데사_충남대.xlsx
                파일명            항목         공급가액
0  23년도 데사_충남대.xlsx  (일반)연구시설•장비비   67626010.0
1  23년도 데사_충남대.xlsx     (특례)학생인건비  203000000.0
2  23년도 데사_충남대.xlsx           간접비  164250000.0
3  23년도 데사_충남대.xlsx          연구수당   73220000.0
4  23년도 데사_충남대.xlsx         연구활동비  282311942.0
5  23년도 데사_충남대.xlsx           인건비   30845620.0
파일명: 23년도 데사_경북대.xlsx
                파일명            항목         공급가액
0  23년도 데사_경북대.xlsx  (일반)연구시설•장비비   37250000.0
1  23년도 데사_경북대.xlsx     (특례)학생인건비  210000000.0
2  23년도 데사_경북대.xlsx           간접비  116025000.0
3  23년도 데사_경북대.xlsx          연구수당   51680000.0
4  23년도 데사_경북대.xlsx         연구활동비  338299956.0
5  23년도 데사_경북대.xlsx           인건비   88561239.0


<ipython-input-14-8ad0d7f27e0c>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')


...

<ipython-input-14-8ad0d7f27e0c>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')
<ipython-input-14-8ad0d7f27e0c>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')


파일명: 23년도 데사_고려대.xlsx
                파일명            항목         공급가액
0  23년도 데사_고려대.xlsx  (일반)연구시설•장비비  498236727.0
1  23년도 데사_고려대.xlsx     (특례)학생인건비  208400000.0
2  23년도 데사_고려대.xlsx           간접비  390501000.0
3  23년도 데사_고려대.xlsx          연구수당   57600000.0
4  23년도 데사_고려대.xlsx         연구재료비    2145000.0
5  23년도 데사_고려대.xlsx         연구활동비  267017653.0
6  23년도 데사_고려대.xlsx           인건비  185087925.0
파일명: 23년도 데사_부경대.xlsx
                파일명            항목         공급가액
0  23년도 데사_부경대.xlsx  (일반)연구시설•장비비  158649060.0
1  23년도 데사_부경대.xlsx     (특례)학생인건비   20000000.0
2  23년도 데사_부경대.xlsx           간접비   51136000.0
3  23년도 데사_부경대.xlsx         연구재료비    7406100.0
4  23년도 데사_부경대.xlsx         연구활동비  117400919.0
5  23년도 데사_부경대.xlsx           인건비   28836650.0


<ipython-input-14-8ad0d7f27e0c>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')
<ipython-input-14-8ad0d7f27e0c>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')


파일명: 23년도 데사_호서대.xlsx
                파일명            항목         공급가액
0  23년도 데사_호서대.xlsx  (일반)연구시설•장비비   79439140.0
1  23년도 데사_호서대.xlsx     (일반)학생인건비  240500000.0
2  23년도 데사_호서대.xlsx           간접비  198516000.0
3  23년도 데사_호서대.xlsx          연구수당   86856000.0
4  23년도 데사_호서대.xlsx         연구재료비   43550000.0
5  23년도 데사_호서대.xlsx         연구활동비  192296540.0
6  23년도 데사_호서대.xlsx           인건비   40330400.0
파일명: 23년도 데사_서울시립.xlsx
                 파일명            항목         공급가액
0  23년도 데사_서울시립.xlsx  (일반)연구시설•장비비   92322630.0
1  23년도 데사_서울시립.xlsx     (특례)학생인건비  339000000.0
2  23년도 데사_서울시립.xlsx           간접비  196880000.0
3  23년도 데사_서울시립.xlsx          연구수당   66809000.0
4  23년도 데사_서울시립.xlsx         연구활동비  186662411.0
5  23년도 데사_서울시립.xlsx           인건비   11933300.0
파일명: 23년도 데사_전남대.xlsx
                파일명            항목         공급가액
0  23년도 데사_전남대.xlsx  (일반)연구시설•장비비   40952230.0
1  23년도 데사_전남대.xlsx     (특례)학생인건비  180000000.0
2  23년도 데사_전남대.xlsx           간접비   61364000.0
3  23년도 데사_전남대.xlsx          연구수당

<ipython-input-14-8ad0d7f27e0c>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')


# [결과3] 24년도 데이터 사이언스 집행 (10건)

In [15]:
import pandas as pd
import glob
import os

# 폴더 경로 및 파일 검색 ("집행" 폴더 내 "22년도" 포함된 파일 찾기)
folder_path = "master/집행/"
file_pattern = os.path.join(folder_path, "*24년도*.xlsx")
file_list = glob.glob(file_pattern)

# 여러 파일에 대한 결과 저장
def process_file(file_path):
    # 엑셀 파일 읽기
    xls = pd.ExcelFile(file_path)
    df = xls.parse(xls.sheet_names[0])  # 첫 번째 시트 사용

    # '항목'과 '공급가액' 열 선택
    df_selected = df.iloc[:, [4, 7]].copy()
    df_selected.columns = ['항목', '공급가액']

    # 병합된 셀 문제 해결: NaN을 앞의 값으로 채움
    df_selected['항목'] = df_selected['항목'].fillna(method='ffill')

    # '공급가액'을 숫자로 변환
    df_selected['공급가액'] = pd.to_numeric(df_selected['공급가액'], errors='coerce')

    # NaN 값 제거 ('공급가액'이 없는 행만 삭제)
    df_selected = df_selected.dropna(subset=['공급가액'])

    # '항목'별 '공급가액' 합산
    result = df_selected.groupby('항목', as_index=False)['공급가액'].sum()

    # 파일명 추가
    result.insert(0, '파일명', os.path.basename(file_path))

    return result

# 모든 파일에 대해 처리 및 출력
for file in file_list:
    result_df = process_file(file)
    print(f"파일명: {os.path.basename(file)}")
    print(result_df)

<ipython-input-15-ee431333bfc2>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')


파일명: 24년도 데사_충남대.xlsx
                파일명            항목         공급가액
0  24년도 데사_충남대.xlsx  (일반)연구시설•장비비   47666010.0
1  24년도 데사_충남대.xlsx     (특례)학생인건비  409000000.0
2  24년도 데사_충남대.xlsx           간접비  157622000.0
3  24년도 데사_충남대.xlsx          연구수당   73220000.0
4  24년도 데사_충남대.xlsx         연구활동비  242782433.0
5  24년도 데사_충남대.xlsx           인건비   61225890.0


<ipython-input-15-ee431333bfc2>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')


파일명: 24년도 데사_고려대.xlsx
                파일명            항목         공급가액
0  24년도 데사_고려대.xlsx  (일반)연구시설•장비비   14245000.0
1  24년도 데사_고려대.xlsx     (특례)학생인건비  798300000.0
2  24년도 데사_고려대.xlsx           간접비  375154000.0
3  24년도 데사_고려대.xlsx          연구수당   27300000.0
4  24년도 데사_고려대.xlsx         연구재료비    2082300.0
5  24년도 데사_고려대.xlsx         연구활동비  263520163.0
6  24년도 데사_고려대.xlsx           인건비  190595172.0


<ipython-input-15-ee431333bfc2>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')
<ipython-input-15-ee431333bfc2>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')


...

<ipython-input-15-ee431333bfc2>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')
<ipython-input-15-ee431333bfc2>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')


파일명: 24년도 데사_카이스트.xlsx
                 파일명         항목          공급가액
0  24년도 데사_카이스트.xlsx  (특례)학생인건비  1.553116e+09
1  24년도 데사_카이스트.xlsx        간접비  2.700000e+08
2  24년도 데사_카이스트.xlsx       연구수당  1.501576e+08
3  24년도 데사_카이스트.xlsx      연구재료비  2.824446e+07
4  24년도 데사_카이스트.xlsx      연구활동비  1.777574e+08
5  24년도 데사_카이스트.xlsx        인건비  1.018527e+08
파일명: 24년도 데사_강원대.xlsx
                파일명            항목         공급가액
0  24년도 데사_강원대.xlsx  (일반)연구시설•장비비   51260000.0
1  24년도 데사_강원대.xlsx     (특례)학생인건비  396000000.0
2  24년도 데사_강원대.xlsx           간접비   78946000.0
3  24년도 데사_강원대.xlsx          연구수당   60000000.0
4  24년도 데사_강원대.xlsx         연구활동비  113533536.0
5  24년도 데사_강원대.xlsx           인건비   81227068.0


<ipython-input-15-ee431333bfc2>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')


파일명: 24년도 데사_서울시립.xlsx
                 파일명         항목         공급가액
0  24년도 데사_서울시립.xlsx  (특례)학생인건비  378000000.0
1  24년도 데사_서울시립.xlsx        간접비  188936000.0
2  24년도 데사_서울시립.xlsx       연구수당   72882000.0
3  24년도 데사_서울시립.xlsx      연구활동비  262351903.0
4  24년도 데사_서울시립.xlsx        인건비    8353310.0
파일명: 24년도 데사_경북대.xlsx
                파일명            항목         공급가액
0  24년도 데사_경북대.xlsx  (일반)연구시설•장비비  354143905.0
1  24년도 데사_경북대.xlsx     (특례)학생인건비  379200000.0
2  24년도 데사_경북대.xlsx           간접비  136136000.0
3  24년도 데사_경북대.xlsx          연구수당  108000000.0
4  24년도 데사_경북대.xlsx         연구활동비  359514889.0
5  24년도 데사_경북대.xlsx           인건비  216540360.0


<ipython-input-15-ee431333bfc2>:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_selected['항목'] = df_selected['항목'].fillna(method='ffill')
